# To do 
1. in the method EmulatedTrials.extract_predictors instead of separate if statements for BMI, HR, and SBP, they should be exctracted in a similar way as lab measurements
2. incorporated the trial_emulation file as a method of the EmulatedTrials class so after predictor extraction and/or setting the trial_predictors_path, one could run EmulatedTrials.emulatetrials(**args)

In [1]:
import os
import itertools
import pandas as pd
import numpy as np
import sys 
import yaml
trial_config=yaml.load(open('./configs/trial_config.yaml'), Loader=yaml.Loader)
from utils_et.utils_et import make_dict_from_table,list_contains
sys.path.insert(0, trial_config['path_params']['cprd_package_paths'])
from emutrials import extract_lab_measurements,EmulatedTrial


from utils_et.yaml_act import yaml_load
from utils.arg_parse import arg_paser
from CPRD.config.spark import spark_init, read_parquet, read_txt, read_csv
from pyspark.sql.types import StructType,StructField, StringType, DateType
from CPRD.functions import tables, merge, cohort_select,risk_prediction, modalities, MedicalDictionary, risk_prediction, predictor_extractor
from pyspark.sql import Window
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import KaplanMeierFitter
import warnings
import copy

warnings.filterwarnings("ignore")

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    
os.environ["JAVA_HOME"] = trial_config['path_params']['java_home']


Bad key savefig.frameon in file /home/mo/anaconda3/envs/cprd/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.4.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file /home/mo/anaconda3/envs/cprd/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.4.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file /home/mo/anaconda3/envs/cprd/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  # a log filename, sys.stdout or sys.stderr')
You probab

# Spark Config

In [2]:
args = dotdict({'params': '/home/shared/mo/HFclusters/dependencies/CPRD/config/config.yaml'}) #path to spark CPRD config file
params = yaml_load(args.params)
params['pyspark']={'temp':'/home/shared/mo/HFclusters/dependencies/sparkDeletable',
                   'pyspark_env':'/home/mo/anaconda3/envs/cprd/bin/python3.7'}
spark_params = params['pyspark']
spark = spark_init(spark_params)
file = params['file_path']

2023-02-17 10:04:57 WARN  Utils:66 - Your hostname, deepmedicine resolves to a loopback address: 127.0.1.1; using 163.1.176.44 instead (on interface enp4s0)
2023-02-17 10:04:57 WARN  Utils:66 - Set SPARK_LOCAL_IP if you need to bind to another address
2023-02-17 10:04:58 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2023-02-17 10:04:59 WARN  SparkConf:66 - Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


# Load cohort data

In [3]:
#cohort selection was previously carried out
data_eval =read_parquet(spark.sqlContext,'/home/shared/yikuan/Mo/analysis/CPC/result.parquet').select(['patid', 'event', 'time', 'label'])
demographic = read_parquet(spark.sqlContext,'/home/shared/yikuan/Mo/demographics.parquet/')

In [4]:
trial_config=yaml_load("./configs/trial_config.yaml")

In [5]:
emulatedtrials=EmulatedTrial(trial_config,demographic,spark)
#emulatedtrials.extract_predictors()

In [12]:
#emulatedtrials.set_trial_predictors_path("/home/shared/mo/HFclusters/cohort_parquets")
#emulatedtrials.generate_path2predictors() #use self.set_trial_predictors_path to change this and conduct emulated trials on previously extracted vars

predictors_paths_dict attribute is now set. Check before running emulatedtrials.
